In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
%load_ext autoreload
%autoreload 2
#useful for displaying wide data frames
pd.set_option('display.max_columns', 50)

In [2]:
#load the data into Pandas dataframes
df_market = pd.read_csv("input/markets.csv")
df_runners = pd.read_csv("input/runners.csv",dtype={'barrier': np.int16,'handicap_weight': np.float16})

#for my simple model, I'm ignoring other columns. I recommend starting with form if you're looking to add features
#df_odds = pd.read_csv("../input/odds.csv")
#df_form = pd.read_csv("../input/forms.csv")
#df_condition = pd.read_csv("../input/conditions.csv")
#df_weather = ("../input/weather.csv")
#df_rider = ("../input/riders.csv")
#df_horse = ("../input/horses.csv")
#df_horse_sex = ("../input/horse_sexes.csv")

In [3]:
##merge the runners and markets data frames
df_runners_and_market = pd.merge(df_runners,df_market,left_on='market_id',right_on='id',how='outer')
#df_runners_and_market.index = df_runners_and_market['id_x'] 
df_runners_and_market = df_runners_and_market.rename(columns={'id_x': 'runner_id'})

In [4]:
df_runners_and_market.head()

,runner_id,collected_at,market_id,position,place_paid,margin,horse_id,trainer_id,rider_id,handicap_weight,number,barrier,blinkers,emergency,form_rating_one,form_rating_two,form_rating_three,last_five_starts,favourite_odds_win,favourite_odds_place,favourite_pool_win,favourite_pool_place,tip_one_win,tip_one_place,tip_two_win,...,tip_four_win,tip_four_place,tip_five_win,tip_five_place,tip_six_win,tip_six_place,tip_seven_win,tip_seven_place,tip_eight_win,tip_eight_place,tip_nine_win,tip_nine_place,id_y,timezone,venue_id,race_number,distance,condition_id,weather_id,total_pool_one_win,total_pool_one_place,total_pool_two_win,total_pool_two_place,total_pool_three_win,total_pool_three_place
0,4.0,2016-06-26 18:54:31.800293,1.0,NaN,1.0,NaN,4.0,4.0,4.0,58.5,4.0,10.0,f,f,82.0,82.0,14.0,x80x2,f,f,f,f,f,NaN,f,...,t,t,t,t,f,t,f,t,f,t,t,t,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
1,10.0,2016-06-26 18:54:31.974395,1.0,NaN,0.0,NaN,10.0,4.0,10.0,56.5,10.0,11.0,f,f,100.0,100.0,18.0,22x35,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
2,5.0,2016-06-26 18:54:31.835329,1.0,NaN,0.0,NaN,5.0,5.0,5.0,56.5,5.0,5.0,f,f,76.0,76.0,0.0,f7,f,f,f,f,f,NaN,f,...,f,f,f,t,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
3,6.0,2016-06-26 18:54:31.873492,1.0,NaN,0.0,NaN,6.0,6.0,6.0,56.5,6.0,12.0,f,f,85.0,85.0,2.0,f6462,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
4,1.0,2016-06-26 18:54:31.746854,1.0,NaN,0.0,NaN,1.0,1.0,1.0,58.5,1.0,8.0,f,f,89.0,89.0,0.0,34x0x,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0


In [6]:
numeric_features = ['position', 'horse_id', 'runner_id', 'market_id','barrier','handicap_weight']
categorical_features = []

#convert to factors
'''for feature in categorical_features:
    df_runners_and_market[feature] = df_runners_and_market[feature].astype(str)
    df_runners_and_market[feature] = df_runners_and_market[feature].replace('nan','0') #have to do this because of a weird random forest bug

    df_features = df_runners_and_market[numeric_features]

for feature in categorical_features:
    encoded_features = pd.get_dummies(df_runners_and_market[feature])
    encoded_features.columns = feature + encoded_features.columns
    df_features = pd.merge(df_features,encoded_features,left_index=True,right_index=True,how='inner') 
'''
df_features = df_runners_and_market[numeric_features]
#turn the target variable into a binary feature: did or did not win
df_features['win'] = False
df_features.loc[df_features['position'] == 1,'win'] = True
#df_features = df_features.reset_index().rename(columns={'id_x': 'horse_id'})
df_features = df_features.reindex(np.random.permutation(df_features.index))
#del df_runners_and_market, encoded_features, df_features['position']

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [89]:
def getInd(race):
    won = race[race.win == True]
    if won.empty:
        wonClass = 8888
    else:
        won = won.index.values[0]
        wonClass = race.index.get_loc(won) 
    return wonClass
.groupby('market_id').apply(getInd)

In [315]:
runnerMap = {}

def createRace(race):
    # we'll need the class of each horse later, which we can get from the indices of this list
    market_id = race.iloc[0,:].market_id
    runnerMap[market_id] = race.runner_id.values
    
    won = race[race.win == True]
    if won.empty:
        won = 8888
        wonClass = 8888
    else:
        won = won.index.values[0]
        wonClass = race.index.get_loc(won) # get what class the winning horse will be    race = race.drop('win', axis=1) # don't need it anymore
    
   
    
    race = race.drop(['horse_id', 'win','position', 'runner_id', 'market_id'], axis=1)
    st = race.stack().to_frame().T #stack it
    
    
    #rename the columns
    # could probably lose this
    st.columns = ['{}_{}'.format(col[1], col[0]) for col in st.columns]
    vals = st.values
    
    #add fake columns to create features for 25 horses
    colsToAdd = (race.shape[1] * 25) - st.shape[1]
    
    ones = (np.ones(colsToAdd) * -999)[:, np.newaxis].T

    withAdded = np.concatenate((vals, ones ), axis=1)
    withAdded = np.append(withAdded, won)
    withAdded = np.append(withAdded, wonClass)
    withAdded = np.append(withAdded, market_id)
    
    
    return withAdded

In [ ]:
df_features.iloc[:2, :].apply(createR)

In [308]:
runnerMap = {}


def createRaceFinal(race):
    # we'll need the class of each horse later, which we can get from the indices of this list
    market_id = race.iloc[0,:].market_id
    runnerMap[market_id] = race.runner_id.values
    
    won = race[race.win == True]
    if won.empty:
        won = 8888
        wonClass = 8888
    else:
        won = won.index.values[0]
        wonClass = race.index.get_loc(won) # get what class the winning horse will be    race = race.drop('win', axis=1) # don't need it anymore
    
    race = race.drop(['horse_id', 'win','position', 'runner_id', 'market_id'], axis=1)
    st = race.stack().to_frame().T #stack it
    
    
    #rename the columns
    # could probably lose this
    st.columns = ['{}_{}'.format(col[1], col[0]) for col in st.columns]
    vals = st.values
    
    #add fake columns to create features for 25 horses
    colsToAdd = (race.shape[1] * 25) - st.shape[1]
    
    ones = (np.ones(colsToAdd) * -999)[:, np.newaxis].T

    withAdded = np.concatenate((vals, ones ), axis=1)
    withAdded = np.append(withAdded, won)
    withAdded = np.append(withAdded, wonClass)
    withAdded = np.append(withAdded, market_id)
    return withAdded

In [ ]:
data = df_features.iloc[:1000,:].groupby('market_id').apply(createRaceFinal)
vals = np.vstack(data.values)
valsMask = vals[:,-2] < 8888
vals = vals[valsMask]
X = vals[:, 0:-3]
y = vals[:, -2]

In [190]:
probs_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,market_id
0,0.246020,0.340441,0.145690,0.004012,0.049847,0.213790,8.565822e-63,1.677282e-04,4.747825e-39,2.626484e-146,2.709727e-237,3.208029e-05,2.242261e-198,0.0,1.137775e-164,3.607386e-223,1.661338e-07,6.167804e-43,0.000000e+00,16.0
1,0.076746,0.257313,0.095382,0.065106,0.133021,0.093158,3.820898e-02,8.343615e-02,1.576272e-01,2.364912e-70,6.581207e-56,1.149445e-129,1.094504e-184,0.0,1.941870e-06,0.000000e+00,6.628875e-225,5.498802e-224,4.279662e-202,17.0
2,0.156870,0.137392,0.090117,0.172983,0.065250,0.089809,3.258109e-02,5.310594e-02,5.703432e-02,7.453298e-02,7.032477e-02,9.544703e-29,4.410844e-120,0.0,1.691873e-47,0.000000e+00,9.083574e-178,0.000000e+00,2.470903e-92,18.0
3,0.060649,0.030329,0.590116,0.056736,0.201685,0.039688,2.047612e-02,1.775849e-37,3.081618e-04,2.970073e-97,1.364171e-186,2.767473e-40,3.259078e-93,0.0,3.200076e-132,0.000000e+00,1.431375e-05,4.840975e-147,0.000000e+00,19.0
4,0.053193,0.043137,0.149693,0.089076,0.028762,0.019537,5.119520e-02,1.976229e-01,1.186156e-01,3.962407e-02,2.095432e-01,6.858533e-28,7.645921e-121,0.0,1.085374e-47,0.000000e+00,1.247967e-175,0.000000e+00,4.319049e-92,20.0


In [71]:
groups = df_features.groupby('market_id')
numGroups = len(groups)
shape = (numGroups, ((df_features.shape[1] - 5) * 25) + 3)

raceData = np.empty(shape)
runnerClassDf = pd.DataFrame(columns=['market_id', 'class', 'runner_id', 'horse_id'])


for (market_id, race), i in zip(groups, np.arange(numGroups)):
    if i > 1000:
        break
    if i % 500 == 0:
        print('{} out of {}'.format(i, numGroups))
    won = race[race.win == True]
    if won.empty:
        continue;
    won = won.index.values[0]
    wonClass = race.index.get_loc(won) # get what class the winning horse will be    race = race.drop('win', axis=1) # don't need it anymore
    
    d = race[['horse_id', 'runner_id', 'market_id']]
    d.loc[:,'class'] = np.arange(d.shape[0])
    runnerClassDf = pd.concat([runnerClassDf, d])

    race = race.drop(['horse_id', 'win','position', 'runner_id', 'market_id'], axis=1)
    st = race.stack().to_frame().T #stack it
    
    
    #rename the columns
    # could probably lose this
    st.columns = ['{}_{}'.format(col[1], col[0]) for col in st.columns]
    vals = st.values
    
    #add fake columns to create features for 25 horses
    colsToAdd = (race.shape[1] * 25) - st.shape[1]
    
    ones = (np.ones(colsToAdd) * -999)[:, np.newaxis].T

    withAdded = np.concatenate((vals, ones ), axis=1)
    withAdded = np.append(withAdded, won)
    withAdded = np.append(withAdded, wonClass)
    withAdded = np.append(withAdded, market_id)
    raceData[i, :] = withAdded

0 out of 7372


//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


500 out of 7372
1000 out of 7372


In [143]:
runnerClassDf.runner_id.unique().shape

(11273,)

In [73]:
colNum = raceMask.shape[1]
zeroMask = (raceData[:,0:] < 1)[:,0]
raceData = raceData[~zeroMask, :]
raceData.shape

(880, 53)

In [74]:
X = raceData[:, 0:-3]
y = raceData[:, -2]

In [185]:
df_features.head()

,position,horse_id,runner_id,market_id,barrier,handicap_weight,win
51567,10.0,8921.0,52279.0,4282.0,9.0,56.5,False
12081,NaN,3999.0,11882.0,964.0,11.0,61.0,False
10280,NaN,7018.0,9795.0,822.0,10.0,56.0,False
9326,8.0,2080.0,9326.0,745.0,7.0,56.0,False
21432,8.0,4367.0,21556.0,1752.0,8.0,57.5,False


In [77]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [197]:
probs = lr.predict_proba(X)

In [199]:
print(probs.shape)

(880, 19)


In [200]:
print(market_ids.shape)

(880,)


In [202]:
colsToAdd = list(np.arange(probs.shape[1]))
colsToAdd.append('market_id')
market_ids = raceData[:, -1].T

probs = np.hstack((probs, market_ids[:, np.newaxis]))
probs_df = pd.DataFrame(probs, columns=colsToAdd)

In [203]:
probs_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,market_id
0,0.246020,3.404408e-01,0.145690,0.004012,0.049847,2.137899e-01,8.565822e-63,1.677282e-04,4.747825e-39,2.626484e-146,2.709727e-237,3.208029e-05,2.242261e-198,0.000000e+00,1.137775e-164,3.607386e-223,1.661338e-07,6.167804e-43,0.000000e+00,16.0
1,0.076746,2.573132e-01,0.095382,0.065106,0.133021,9.315807e-02,3.820898e-02,8.343615e-02,1.576272e-01,2.364912e-70,6.581207e-56,1.149445e-129,1.094504e-184,0.000000e+00,1.941870e-06,0.000000e+00,6.628875e-225,5.498802e-224,4.279662e-202,17.0
2,0.156870,1.373917e-01,0.090117,0.172983,0.065250,8.980935e-02,3.258109e-02,5.310594e-02,5.703432e-02,7.453298e-02,7.032477e-02,9.544703e-29,4.410844e-120,0.000000e+00,1.691873e-47,0.000000e+00,9.083574e-178,0.000000e+00,2.470903e-92,18.0
3,0.060649,3.032919e-02,0.590116,0.056736,0.201685,3.968752e-02,2.047612e-02,1.775849e-37,3.081618e-04,2.970073e-97,1.364171e-186,2.767473e-40,3.259078e-93,0.000000e+00,3.200076e-132,0.000000e+00,1.431375e-05,4.840975e-147,0.000000e+00,19.0
4,0.053193,4.313738e-02,0.149693,0.089076,0.028762,1.953690e-02,5.119520e-02,1.976229e-01,1.186156e-01,3.962407e-02,2.095432e-01,6.858533e-28,7.645921e-121,0.000000e+00,1.085374e-47,0.000000e+00,1.247967e-175,0.000000e+00,4.319049e-92,20.0
5,0.057950,1.170077e-01,0.109014,0.070390,0.096770,1.166512e-01,1.110342e-01,1.915997e-01,1.295805e-01,1.823045e-70,9.044495e-56,5.841328e-130,7.352231e-185,0.000000e+00,2.501866e-06,0.000000e+00,4.120410e-227,6.003204e-224,2.670834e-202,21.0
6,0.077082,2.373712e-02,0.077700,0.008977,0.033996,4.760473e-02,1.686958e-02,2.873847e-02,2.312176e-01,1.670386e-02,5.813759e-02,4.537407e-02,1.685703e-02,3.160878e-01,9.174509e-04,0.000000e+00,6.361807e-123,4.938920e-12,6.267438e-11,22.0
7,0.091748,5.062903e-02,0.079400,0.057918,0.048545,1.568736e-01,1.632511e-01,7.410848e-02,6.485819e-02,8.335420e-02,7.919667e-02,5.011722e-02,1.134808e-40,0.000000e+00,3.374368e-10,0.000000e+00,2.658301e-185,2.063698e-122,4.783228e-19,23.0
8,0.055269,5.500486e-02,0.052259,0.082606,0.147029,2.662211e-01,4.091512e-02,1.612524e-02,3.823726e-02,4.830854e-02,1.209344e-01,2.281763e-02,3.399933e-02,2.027206e-02,2.990200e-65,0.000000e+00,5.764777e-147,9.770307e-110,9.487016e-07,24.0
9,0.116707,6.002019e-02,0.190804,0.274227,0.062117,1.368179e-02,7.729681e-02,6.068624e-02,4.501020e-02,3.934018e-02,6.010908e-02,2.733796e-28,8.284729e-120,0.000000e+00,3.067492e-47,0.000000e+00,1.451422e-176,0.000000e+00,6.730663e-92,25.0


In [80]:
market_ids = raceData[:,-1].T

In [138]:
probs_re = probs_df.set_index('market_id').stack().reset_index().rename(columns={0:'prob', 'level_1':'class'})

In [142]:
runnerClassDf.shape

(11273, 4)

In [147]:
probs_re['class'] = probs_re['class'].astype(float)

In [174]:
classAndProb = probs_re.merge(runnerClassDf, on=['market_id', 'class'])

In [156]:
all_odds = pd.read_csv("input/odds.csv")

In [178]:
def addOdds(classAndProb, all_odds):
    df_odds = all_odds[all_odds['runner_id'].isin(classAndProb.runner_id)]
    average_win_odds = df_odds.groupby(['runner_id'])['odds_one_win'].mean()
    average_win_odds[average_win_odds == 0] = np.nan

    average_win_odds = pd.DataFrame(average_win_odds).reset_index().rename(columns={'odds_one_win':'odds'})
    average_win_odds = average_win_odds.dropna(subset=['odds'])

    return classAndProb.merge(average_win_odds, on='runner_id')

classAndProb = addOdds(classAndProb, all_odds)

In [187]:
classAndProb = classAndProb.merge(df_features[['runner_id', 'win']], on='runner_id')

In [188]:
def placeBets(df):
    investment = 0
    payout = 0
    bets = 0
    wins = 0
    for index, row in df.iterrows():
        # if we predicted the probability of a win that's greater than the odds, we bet
        if row['prob'] > (1/row['odds']):
            investment +=1
            bets +=1
            if (row['win']):
                payout += row['odds']
                wins +=1
                
    investment_return = round((payout - investment)/investment*100,2)
    print('Total bets: {}'.format(bets))
    print('Total wins: {}'.format(wins))
    return investment_return

In [189]:
placeBets(classAndProb)

Total bets: 3975
Total wins: 357


24.99

In [16]:
from runRaces import runRaces
from sklearn.linear_model import LogisticRegression

p = runRaces(df_f, LogisticRegression(), allOdds,n_trials=3)

trial 0
position           7651
market_id             0
barrier               0
handicap_weight       0
win                   0
dtype: int64


/Users/ryankresse/Documents/py/horses/runRaces.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_test['winProb'] = predictions


Total bets: 10200
Total wins: 250
trial 1
position           7571
market_id             0
barrier               0
handicap_weight       0
win                   0
dtype: int64
Total bets: 10293
Total wins: 297
trial 2
position           7313
market_id             0
barrier               0
handicap_weight       0
win                   0
dtype: int64
Total bets: 10048
Total wins: 259
mean payout: -27.929999999999996, std payout: 5.752257527846494


In [15]:
%reload_ext autoreload